In [1]:
"""
Create region coverage maps with contextily basemap
Shows visited tiles in orange, unvisited tiles in grey
Uses OpenStreetMap basemap via contextily
Works entirely in Web Mercator (EPSG:3857) for plotting
"""

import pandas as pd
import geopandas as gpd
import osmnx as ox
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Polygon
from shapely.geometry import Point, box
import contextily as cx

def tile_to_lon_lat(tile_x, tile_y, zoom):
    """Convert tile coordinates to longitude/latitude (center of tile)"""
    n = 2.0 ** zoom
    lon = (tile_x + 0.5) / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * (tile_y + 0.5) / n)))
    lat = math.degrees(lat_rad)
    return lon, lat

def tile_to_bbox(tile_x, tile_y, zoom):
    """Convert tile to bounding box coordinates"""
    n = 2.0 ** zoom
    west = tile_x / n * 360.0 - 180.0
    east = (tile_x + 1) / n * 360.0 - 180.0
    
    lat_rad_north = math.atan(math.sinh(math.pi * (1 - 2 * tile_y / n)))
    north = math.degrees(lat_rad_north)
    
    lat_rad_south = math.atan(math.sinh(math.pi * (1 - 2 * (tile_y + 1) / n)))
    south = math.degrees(lat_rad_south)
    
    return west, east, south, north

def create_region_maps(region, squadrat_file, squadratinho_file, output_dir='output'):
    """
    Create two side-by-side maps showing region coverage at zoom 14 and 17
    
    Parameters:
    -----------
    region : str
        Name of the region (e.g., 'Geneva', 'Vaud', 'Basel')
    squadrat_file : str
        Path to squadrat grid mask CSV file
    squadratinho_file : str
        Path to squadratinho grid mask CSV file
    output_dir : str
        Output directory for saving the maps
    """
    
    print(f"Creating {region} coverage maps...")
    
    # Download region boundary
    print(f"Downloading {region} boundary from OpenStreetMap...")
    if region == "Canton of Geneva":
        region_gdf = ox.geocode_to_gdf(
            {'state': 'Geneva'}, 
            by_osmid=False,
            which_result=1
        )
    else:
        region_gdf = ox.geocode_to_gdf(f'{region}, Switzerland', which_result=1)
    
    # Convert to Web Mercator for plotting
    region_gdf_mercator = region_gdf.to_crs(epsg=3857)
    region_geom = region_gdf.geometry.iloc[0]
    region_geom_mercator = region_gdf_mercator.geometry.iloc[0]
    
    # Get region bounds in lat/lon for calculations
    bounds = region_geom.bounds
    lon_min, lat_min, lon_max, lat_max = bounds
    
    # Add padding
    lon_padding = (lon_max - lon_min) * 0.1
    lat_padding = (lat_max - lat_min) * 0.1
    lon_min_padded = lon_min - lon_padding
    lon_max_padded = lon_max + lon_padding
    lat_min_padded = lat_min - lat_padding
    lat_max_padded = lat_max + lat_padding
    
    # ========== LOAD AND PROCESS SQUADRAT (Zoom 14) ==========
    print("\nProcessing Squadrat (Zoom 14) data...")
    
    df_squadrat = pd.read_csv(squadrat_file)
    
    # Convert tiles to geometry
    tiles_squadrat = []
    for _, row in df_squadrat.iterrows():
        lon, lat = tile_to_lon_lat(row['tile_x'], row['tile_y'], row['zoom'])
        point = Point(lon, lat)
        west, east, south, north = tile_to_bbox(row['tile_x'], row['tile_y'], row['zoom'])
        tiles_squadrat.append({
            'tile_x': row['tile_x'],
            'tile_y': row['tile_y'],
            'zoom': row['zoom'],
            'visited': row['inside_polygon'] == 1,
            'geometry': point,
            'west': west, 'east': east, 'south': south, 'north': north
        })
    
    gdf_squadrat = gpd.GeoDataFrame(tiles_squadrat, crs='EPSG:4326')
    gdf_squadrat['in_region'] = gdf_squadrat.within(region_geom)
    
    # Convert to Web Mercator for plotting
    gdf_squadrat_mercator = gdf_squadrat.to_crs(epsg=3857)
    
    # Calculate coverage
    tiles_in_region = gdf_squadrat[gdf_squadrat['in_region']]
    visited_in_region = (tiles_in_region['visited']).sum()
    coverage_squadrat = (visited_in_region / len(tiles_in_region) * 100) if len(tiles_in_region) > 0 else 0
    
    print(f"Squadrat - Total in {region}: {len(tiles_in_region)}, Visited: {visited_in_region}, Coverage: {coverage_squadrat:.2f}%")
    
    # ========== LOAD AND PROCESS SQUADRATINHO (Zoom 17) ==========
    print("\nProcessing Squadratinho (Zoom 17) data...")
    
    df_squadratinho = pd.read_csv(squadratinho_file)
    
    # Convert tiles to geometry
    tiles_squadratinho = []
    for _, row in df_squadratinho.iterrows():
        lon, lat = tile_to_lon_lat(row['tile_x'], row['tile_y'], row['zoom'])
        point = Point(lon, lat)
        west, east, south, north = tile_to_bbox(row['tile_x'], row['tile_y'], row['zoom'])
        tiles_squadratinho.append({
            'tile_x': row['tile_x'],
            'tile_y': row['tile_y'],
            'zoom': row['zoom'],
            'visited': row['inside_polygon'] == 1,
            'geometry': point,
            'west': west, 'east': east, 'south': south, 'north': north
        })
    
    gdf_squadratinho = gpd.GeoDataFrame(tiles_squadratinho, crs='EPSG:4326')
    gdf_squadratinho['in_region'] = gdf_squadratinho.within(region_geom)
    
    # Convert to Web Mercator for plotting
    gdf_squadratinho_mercator = gdf_squadratinho.to_crs(epsg=3857)
    
    # Calculate coverage
    tiles_in_region = gdf_squadratinho[gdf_squadratinho['in_region']]
    visited_in_region = (tiles_in_region['visited']).sum()
    coverage_squadratinho = (visited_in_region / len(tiles_in_region) * 100) if len(tiles_in_region) > 0 else 0
    
    print(f"Squadratinho - Total in {region}: {len(tiles_in_region)}, Visited: {visited_in_region}, Coverage: {coverage_squadratinho:.2f}%")
    
    # ========== CREATE PLOTS ==========
    print("\nCreating plots...")
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 12))
    
    # ========== SQUADRAT PLOT (LEFT) ==========
    ax1.set_title(f'Squadrat Grid (Zoom 14) - {region}', fontsize=16, fontweight='bold', pad=15)
    
    # Plot tiles in Web Mercator coordinates
    for idx, row in gdf_squadrat_mercator.iterrows():
        # Get bbox in original lat/lon
        west_ll, east_ll = gdf_squadrat.loc[idx, 'west'], gdf_squadrat.loc[idx, 'east']
        south_ll, north_ll = gdf_squadrat.loc[idx, 'south'], gdf_squadrat.loc[idx, 'north']
        
        # Create bbox in lat/lon then convert to mercator
        bbox_ll = box(west_ll, south_ll, east_ll, north_ll)
        bbox_gdf = gpd.GeoDataFrame([1], geometry=[bbox_ll], crs='EPSG:4326')
        bbox_merc = bbox_gdf.to_crs(epsg=3857).geometry.iloc[0]
        
        minx, miny, maxx, maxy = bbox_merc.bounds
        
        # Determine color and style
        if not row['visited'] and row['in_region']:
            facecolor = 'lightgrey'
            edgecolor = '#5B4B8A'
            alpha = 0.6
            linewidth = 1.0
            zorder = 3
        elif not row['visited'] and not row['in_region']:
            continue
        elif row['visited'] and row['in_region']:
            facecolor = 'tab:orange'
            edgecolor = '#5B4B8A'
            alpha = 0.8
            linewidth = 2.5
            zorder = 5
        else:
            facecolor = '#FFD9B3'
            edgecolor = '#5B4B8A'
            alpha = 0.7
            linewidth = 2.0
            zorder = 4
        
        rect = patches.Rectangle((minx, miny), maxx - minx, maxy - miny,
                                linewidth=linewidth, edgecolor=edgecolor,
                                facecolor=facecolor, alpha=alpha, zorder=zorder)
        ax1.add_patch(rect)
    
    # Plot region boundary
    if region_geom_mercator.geom_type == 'MultiPolygon':
        for geom in region_geom_mercator.geoms:
            xs, ys = geom.exterior.coords.xy
            ax1.plot(xs, ys, color='black', linewidth=3, zorder=10)
    else:
        xs, ys = region_geom_mercator.exterior.coords.xy
        ax1.plot(xs, ys, color='black', linewidth=3, zorder=10)
    
    # Add contextily basemap (no crs parameter needed - defaults to EPSG:3857)
    cx.add_basemap(ax1, source=cx.providers.OpenStreetMap.Mapnik, alpha=0.5, zorder=0)
    
    # Add coverage text
    tiles_in_region_squadrat = gdf_squadrat[gdf_squadrat['in_region']]
    visited_in_region_squadrat = (tiles_in_region_squadrat['visited']).sum()
    ax1.text(0.02, 0.98, f'Coverage: {coverage_squadrat:.2f}%\nTotal tiles in {region}: {len(tiles_in_region_squadrat):,}\nVisited in {region}: {visited_in_region_squadrat:,}',
            transform=ax1.transAxes, fontsize=12, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.9), zorder=11)
    
    ax1.set_xlabel('X (meters)', fontsize=12)
    ax1.set_ylabel('Y (meters)', fontsize=12)
    
    # ========== SQUADRATINHO PLOT (RIGHT) ==========
    ax2.set_title(f'Squadratinho Grid (Zoom 17) - {region}', fontsize=16, fontweight='bold', pad=15)
    
    # Plot tiles in Web Mercator coordinates
    for idx, row in gdf_squadratinho_mercator.iterrows():
        # Get bbox in original lat/lon
        west_ll, east_ll = gdf_squadratinho.loc[idx, 'west'], gdf_squadratinho.loc[idx, 'east']
        south_ll, north_ll = gdf_squadratinho.loc[idx, 'south'], gdf_squadratinho.loc[idx, 'north']
        
        # Create bbox in lat/lon then convert to mercator
        bbox_ll = box(west_ll, south_ll, east_ll, north_ll)
        bbox_gdf = gpd.GeoDataFrame([1], geometry=[bbox_ll], crs='EPSG:4326')
        bbox_merc = bbox_gdf.to_crs(epsg=3857).geometry.iloc[0]
        
        minx, miny, maxx, maxy = bbox_merc.bounds
        
        # Determine color and style
        if not row['visited'] and row['in_region']:
            facecolor = 'lightgrey'
            edgecolor = '#5B4B8A'
            alpha = 0.6
            linewidth = 0.5
            zorder = 3
        elif not row['visited'] and not row['in_region']:
            continue
        elif row['visited'] and row['in_region']:
            facecolor = 'tab:orange'
            edgecolor = '#5B4B8A'
            alpha = 0.8
            linewidth = 1.5
            zorder = 5
        else:
            facecolor = '#FFD9B3'
            edgecolor = '#5B4B8A'
            alpha = 0.7
            linewidth = 1.0
            zorder = 4
        
        rect = patches.Rectangle((minx, miny), maxx - minx, maxy - miny,
                                linewidth=linewidth, edgecolor=edgecolor,
                                facecolor=facecolor, alpha=alpha, zorder=zorder)
        ax2.add_patch(rect)
    
    # Plot region boundary
    if region_geom_mercator.geom_type == 'MultiPolygon':
        for geom in region_geom_mercator.geoms:
            xs, ys = geom.exterior.coords.xy
            ax2.plot(xs, ys, color='black', linewidth=3, zorder=10)
    else:
        xs, ys = region_geom_mercator.exterior.coords.xy
        ax2.plot(xs, ys, color='black', linewidth=3, zorder=10)
    
    # Add contextily basemap
#    cx.add_basemap(ax2, source=cx.providers.OpenStreetMap.Mapnik, alpha=0.5, zorder=0)
    cx.add_basemap(ax2, source=cx.providers.Stamen.Terrain, alpha=0.5, zorder=0)
    
    # Add coverage text
    tiles_in_region_squadratinho = gdf_squadratinho[gdf_squadratinho['in_region']]
    visited_in_region_squadratinho = (tiles_in_region_squadratinho['visited']).sum()
    ax2.text(0.02, 0.98, f'Coverage: {coverage_squadratinho:.2f}%\nTotal tiles in {region}: {len(tiles_in_region_squadratinho):,}\nVisited in {region}: {visited_in_region_squadratinho:,}',
            transform=ax2.transAxes, fontsize=12, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.9), zorder=11)
    
    ax2.set_xlabel('X (meters)', fontsize=12)
    ax2.set_ylabel('Y (meters)', fontsize=12)
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/{region}_basemap.png', dpi=150, bbox_inches='tight')
    print(f"\nSaved: {output_dir}/{region}_basemap.png")
    plt.close()
    
    print("\nDone! Map created successfully.")


# Main execution
if __name__ == "__main__":
    
    squadrat_file = 'output/squadrat_grid_mask_Nyon.csv'
    squadratinho_file = 'output/squadratinho_grid_mask_Nyon.csv'
    
    create_region_maps(
        region='Geneva',
        squadrat_file=squadrat_file,
        squadratinho_file=squadratinho_file,
        output_dir='figure'
    )

/Users/epauthen/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Creating Geneva coverage maps...

Processing Squadrat (Zoom 14) data...
Squadrat - Total in Geneva: 6, Visited: 5, Coverage: 83.33%

Processing Squadratinho (Zoom 17) data...
Squadratinho - Total in Geneva: 409, Visited: 136, Coverage: 33.25%

Creating plots...

Saved: figure/Geneva_basemap.png

Done! Map created successfully.
